## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Barrow,US,71.2906,-156.7887,42.82,91,90,13.80,overcast clouds
1,1,Norman Wells,CA,65.2820,-126.8329,66.24,45,20,8.05,few clouds
2,2,Bengkulu,ID,-3.8004,102.2655,74.89,87,0,5.50,clear sky
3,3,Mataura,NZ,-46.1927,168.8643,46.47,92,100,3.22,light rain
4,4,Skoropuskovskiy,RU,56.3667,38.1667,68.56,55,89,3.36,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 60


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
6,6,Busselton,AU,-33.6500,115.3333,58.08,66,5,20.22,clear sky
12,12,Lokomotivnyy,RU,53.0118,60.5684,58.69,77,69,6.26,broken clouds
23,23,Thompson,CA,55.7435,-97.8558,59.16,51,75,13.80,broken clouds
28,28,Mar Del Plata,AR,-38.0023,-57.5575,57.09,60,90,3.00,overcast clouds
32,32,Rawson,AR,-43.3002,-65.1023,51.91,36,5,21.43,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             189
City                   189
Country                186
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Weather Description    189
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
import numpy as np

preferred_cities_df["Country"].replace(" ", np.nan)

#preferred_cities_df.isnull().sum()

new_df = preferred_cities_df.dropna()

new_df.count()

Unnamed: 0             186
City                   186
Country                186
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Weather Description    186
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
6,Busselton,AU,58.08,clear sky,-33.6500,115.3333,
12,Lokomotivnyy,RU,58.69,broken clouds,53.0118,60.5684,
23,Thompson,CA,59.16,broken clouds,55.7435,-97.8558,
28,Mar Del Plata,AR,57.09,overcast clouds,-38.0023,-57.5575,
32,Rawson,AR,51.91,clear sky,-43.3002,-65.1023,
37,Carnarvon,AU,53.67,clear sky,-24.8667,113.6333,
52,Tuktoyaktuk,CA,50.00,light rain,69.4541,-133.0374,
55,Kodiak,US,51.31,broken clouds,57.7900,-152.4072,
71,Hwange,ZW,59.02,clear sky,-18.3693,26.5019,
92,Constitucion,CL,53.58,scattered clouds,-35.3333,-72.4167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    #add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()    

Observatory Guest House
Hotel not found... skipping.
Burntwood Hotel
Gran Hotel Mar del Plata
Hosteria Sampedro
Hospitality Carnarvon
End of The Road Ltd.
Best Western Kodiak Inn And Convention Center
Hwange Colliery Guest House
Hotel Boutique Las Azucenas
Shellharbour Resort & Conference Centre
Sjybrygga fiskecamp
Hotel Arctic
Big Tree
Grand Hotel Río Cuarto
39 On Nile Guest House
Vandrehuset
Park Pobedy
Bushido Lodge
Gostinitsa Ki'ileekh
Lunny Kot Guest House
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Blue Bay Lodge
Royal Plaza
dyd -mar Hotel
Hospitality Esperance
Tu Casa
International Lodge Motel
The George
Scenic Hotel Southern Cross
Hostal Entre Rios
Hotel Olafsvik
Northern Light Inn & Max's Restaurant
Hostal Las Lilas
Super Leisure Hotel
Westmark Sitka
Zapolyar'ye, Gostinitsa
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Narsaq Kayak Hostel
Hotel not found... skipping.
Desert Palms Alice Springs


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
6,Busselton,AU,58.08,clear sky,-33.6500,115.3333,Observatory Guest House
12,Lokomotivnyy,RU,58.69,broken clouds,53.0118,60.5684,
23,Thompson,CA,59.16,broken clouds,55.7435,-97.8558,Burntwood Hotel
28,Mar Del Plata,AR,57.09,overcast clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
32,Rawson,AR,51.91,clear sky,-43.3002,-65.1023,Hosteria Sampedro
37,Carnarvon,AU,53.67,clear sky,-24.8667,113.6333,Hospitality Carnarvon
52,Tuktoyaktuk,CA,50.00,light rain,69.4541,-133.0374,End of The Road Ltd.
55,Kodiak,US,51.31,broken clouds,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center
71,Hwange,ZW,59.02,clear sky,-18.3693,26.5019,Hwange Colliery Guest House
92,Constitucion,CL,53.58,scattered clouds,-35.3333,-72.4167,Hotel Boutique Las Azucenas


In [52]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df["Hotel Name"].replace(" ", np.nan)


clean_hotel_df = hotel_df.dropna()



In [53]:
# 8a. Create the output File (CSV)
output_data_file = ("Vacation_Search\WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [54]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """"
    <dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
    <dt>Weather Description</dt><dd>{Weather Description}<dd>
    </dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))